In [ ]:
import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import os
from PIL import Image
from io import BytesIO
import time

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11129153428305754079
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14619377664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8918741222912546032
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
def mamon_videoFightModel2(tf,wight='/content/mamonbest947oscombo-drive.h5'):
    layers = tf.keras.layers
    models = tf.keras.models
    losses = tf.keras.losses
    optimizers = tf.keras.optimizers
    metrics = tf.keras.metrics
    num_classes = 2
    cnn = models.Sequential()




    input_shapes=(160,160,3)
    np.random.seed(1234)
    vg19 = tf.keras.applications.vgg19.VGG19
    base_model = vg19(include_top=False,weights='imagenet',input_shape=(160, 160,3))


    cnn = models.Sequential()
    cnn.add(base_model)
    cnn.add(layers.Flatten())
    model = models.Sequential()

    model.add(layers.TimeDistributed(cnn,  input_shape=(30, 160, 160, 3)))
    model.add(layers.LSTM(30 , return_sequences= True))

    model.add(layers.TimeDistributed(layers.Dense(90)))
    model.add(layers.Dropout(0.1))

    model.add(layers.GlobalAveragePooling1D())

    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(num_classes, activation="sigmoid"))

    adam = optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.load_weights(wight)
    rms = optimizers.RMSprop()

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

    return model

In [ ]:
print(tf. __version__)


2.18.0


In [ ]:
import numpy as np
from skimage.transform import resize
np.random.seed(1234)
model22 = mamon_videoFightModel2(tf)

In [ ]:
def video_mamonreader(cv2, filename):
    frames = np.zeros((30, 160, 160, 3), dtype=float)
    i = 0
    print(frames.shape)
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval, frame = vc.read()
        if rval:
            frm = resize(frame, (160, 160, 3))
            frm = np.expand_dims(frm, axis=0)
            if (np.max(frm) > 1):
                frm = frm / 255.0
            frames[i][:] = frm
            i += 1
            print("reading video")
            while i < 30:
                rval, frame = vc.read()
                if rval:
                    frm = resize(frame, (160, 160, 3))
                    frm = np.expand_dims(frm, axis=0)
                    if (np.max(frm) > 1):
                        frm = frm / 255.0
                    frames[i][:] = frm
                    i += 1
                else:
                    # Handle situation where video frames end unexpectedly
                    break

            return frames
        else:
            print(f"Error reading video file: {filename}")
            return None
    else:
        print(f"Error opening video file: {filename}")
        return None

In [ ]:
def pred_fight(model,video,acuracy=0.9):
    pred_test = model.predict(video)
    if pred_test[0][1] >=acuracy:
        return True , pred_test[0][1]
    else:
        return False , pred_test[0][1]

In [ ]:
def main_fight(vidoss):
    vid = video_mamonreader(cv2,vidoss)
    datav = np.zeros((1, 30, 160, 160, 3), dtype=float)
    datav[0][:][:] = vid
    millis = int(round(time.time() * 1000))
    print(millis)
    f , precent = pred_fight(model22,datav,acuracy=0.65)
    millis2 = int(round(time.time() * 1000))
    print(millis2)
    res_mamon = {'fight':f , 'precentegeoffight':str(precent)}
    res_mamon['processing_time'] =  str(millis2-millis)
    return res_mamon

In [ ]:
res = main_fight('/content/fi030.mp4')
res

(30, 160, 160, 3)
reading video
1738263054095
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1738263054473


{'fight': True, 'precentegeoffight': '0.9915508', 'processing_time': '378'}

In [ ]:
res = main_fight('/content/nofi002.mp4')
res

(30, 160, 160, 3)
reading video
1730320336098
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1730320348208


{'fight': False, 'precentegeoffight': '0.4744618', 'processing_time': '12110'}

In [ ]:
res = main_fight('/content/nofi020.mp4')
res

(30, 160, 160, 3)
reading video
1730320351186
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1730320361448


{'fight': False, 'precentegeoffight': '0.020639', 'processing_time': '10262'}